In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Wed Oct 2 10:38:36 2019

@author: shlokmehrotra
"""
#essential libraries
from PIL import Image
import numpy as np
import csv
import cv2
from numpy import linspace
from pylab import plot,show,hist,figure,title
from scipy.stats import norm
from scipy import optimize
import scipy.optimize as opt
from scipy.optimize import curve_fit
from scipy import asarray as ar
from mpl_toolkits import mplot3d

In [2]:
def read_tiff1(path, time_intervals, stack_size, xlen, ylen):
    """
    path - Path to the multipage-tiff file
    n_images - Number of pages in the tiff file
    """
    img = Image.open(path)
    #meta data of tiff modified such that only grayscale image values appear -> previously in the thousands
    print(img.height, img.width)
    num_stacks = time_intervals
    #num_stacks = int(n_images/stack_size) -> given total amount of frames
    images = []
    for i in range(num_stacks):
        val = []
        for j in range(stack_size):
            
            try:
                img.seek(i*time_intervals + j)
                pixval = [img.getdata()]
                val.append(np.array(pixval).reshape(xlen, ylen))
                
            except EOFError:
                # Not enough frames in img
                break
        print(i ," is done")
        images.append(val)
    return(np.array(images))

In [3]:
def identifySpotLoc(data, time_intervals, stack_size, xlen, ylen):
    brightest = []
    frame_area = xlen * ylen
    #Non-Local Means Denoising
    data = np.array(data)
    finarr = []
    newarr = []
    for i in range(time_intervals): 
        newarr = []
        for k in range(stack_size):
            #print(data[0][0].shape)
            #print(data)
            newarr.append(cv2.fastNlMeansDenoising(np.uint8(data[i][k]), None, 10, 13))
        finarr.append(newarr)
    finarr = np.array(finarr)
    
    for i in range(time_intervals):
        val = []
        for j in range(xlen):
            for p in range(ylen):     
                pix_bright = []
                for k in range(stack_size):
                    # ------ CHANGED pix_bright.append(finarr[i][k][j][p])
                    weight = int(int(stack_size/2) - np.absolute(np.round(stack_size/2)-k-1)+1)
                    #weight = stack_size - (np.absolute([int(stack_size/2 - k)]))
                    pix_bright.append(weight * finarr[i][k][j][p])
                
                #print(len(pix_bright))
                #print(pix_bright)
                #pix_bright = np.array(pix_bright)
                #weight = stack_size - (np.absolute([int(stack_size/2) - k])
                weightsum = int((stack_size/2) * ((stack_size/2)+1)/2)
                val.append(np.round(np.sum(pix_bright)/weightsum))
                
                #-----CHANGEDval.append(int(np.round((max(pix_bright) + sum(pix_bright)/len(pix_bright))/2)))
            #val.append(np.mean(pix_bright))
        

            #if(pix_bright[0] != max(pix_bright)):
            #    print("yay ", pix_bright[0], " ", max(pix_bright))
            #print("ya so this", pix_bright, pix_bright[0])
            brightest.append(val)
    fin_data = []
    brightarr = np.array(brightest)
    for i in range(time_intervals):
        arr = brightarr[i]
        arr = arr.reshape(xlen, ylen)
        fin_data.append(arr)
        print(arr.shape)
        #print(len(brightarr[i]))
    #brightarr.reshape((xlen, ylen))
    cv2.imwrite("testtiff.tif", fin_data[0])
    fin_data = np.array(fin_data)
    print("data shape ting", fin_data.shape)
    return(fin_data)

In [4]:
def findSpots(brightarr,time_frames, min_pixel_intensity):
    
    spotlocs = []
    for i in range(time_frames):
        # Setup SimpleBlobDetector parameters.
        params = cv2.SimpleBlobDetector_Params()

        # Change thresholds
        params.minThreshold = min_pixel_intensity
        params.maxThreshold = 255

        #looking for spots that tend towards lighter colors
        params.blobColor = 255

        # Filter by Area.
        params.filterByArea = False
        params.minArea = 10
        params.maxArea = 2500

        # Filter by Circularity (Future Reference if needed)
        params.filterByCircularity = False
        #params.minCircularity = 0.1

        # Filter by Convexity (Future Reference if needed)
        params.filterByConvexity = False
        #params.minConvexity = 0.87

        # Filter by Inertia (Future Reference if needed)
        params.filterByInertia = False
        #params.minInertiaRatio = 0.01

        # Create a detector with the parameters
        detector = cv2.SimpleBlobDetector_create(params)


        # Detect blobs.
        keypoints = detector.detect(brightarr[i])
        for i in range(len(keypoints)):
            x = (keypoints[i].pt[0])
            y = (keypoints[i].pt[1])
            spotlocs.append((x, y))
        im = brightarr[i]
        im_with_keypoints = cv2.drawKeypoints(im, keypoints, np.array([]), (0,255,0), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
        cv2.imwrite("testtiffwspots1.tif", im_with_keypoints)
    return(spotlocs)

In [5]:
def findSpotsPerLayer(data, time_frames, stacks, min_pixel_intensity):
    spots = []
    print(np.array(data[0][0]).shape)
    for i in range(time_frames):
        totalspots = []
        for j in range(stacks):
            spotlocs = []
            # Setup SimpleBlobDetector parameters.
            params = cv2.SimpleBlobDetector_Params()

            # Change thresholds
            params.minThreshold = min_pixel_intensity
            params.maxThreshold = 255

            #looking for spots that tend towards lighter colors
            params.blobColor = 255

            # Filter by Area.
            params.filterByArea = False
            params.minArea = 10
            params.maxArea = 2500

            # Filter by Circularity (Future Reference if needed)
            params.filterByCircularity = False
            #params.minCircularity = 0.1

            # Filter by Convexity (Future Reference if needed)
            params.filterByConvexity = False
            #params.minConvexity = 0.87

            # Filter by Inertia (Future Reference if needed)
            params.filterByInertia = False
            #params.minInertiaRatio = 0.01

            # Create a detector with the parameters
            detector = cv2.SimpleBlobDetector_create(params)


            # Detect blobs.
            try:
                keypoints = detector.detect(data[i][j]) #data[time intervals][stack layers]
            except:
                print("The error is in the goose index ", i, " ", j)
            for k in range(len(keypoints)):
                x = (keypoints[k].pt[0])
                y = (keypoints[k].pt[1])
                #print(x, y)
                spotlocs.append((x, y))
            #print("next")
            spotlocs = np.array(spotlocs)
            totalspots.append(spotlocs)
            #im = data[i][j]
            #im_with_keypoints = cv2.drawKeypoints(im, keypoints, np.array([]), (0,255,0), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
        totalspots = np.array(totalspots)
        spots.append(totalspots)
    #print(np.array(data[0][0]).shape)
    spots = np.array(spots)
    with open('FoundSpots.csv', 'w') as writeFile:
        writer = csv.writer(writeFile)
        writer.writerows(spots)
        
    return(spots)
#Spots Dimensions:
#1st layer -> time frames
#2nd layer - > Z stack
#3rd layer -> things found in that stack.


###  The following is to process the spots such that the spots can be found.
Run time takes a while for following cell



In [6]:
#readtiff1(path, time_interval, stack_size)
#data = read_tiff1("Series011.tif", 29, 19)
time_frames = 3
z_stack = 19
data = read_tiff1("Series011.tif", time_frames, z_stack, 768, 768)


with open('imageData.csv', 'w') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(data)
writeFile.close()
print(data.size)
print(data[0].size) #At time = 0;
print(data[0][0].size) #At time = 0 and z stack of 0


#identifySpotLoc(time_intervals, stacks, xlen, ylen)
#spotFinds = identifySpotLoc(data, 29, 19, 768, 768)
compressed = identifySpotLoc(data, time_frames, z_stack, 768, 768)
print(compressed[0][:10])

print("Size of spotFinds: ", len(compressed)) #should be 768 * 768 * 29
print("spotarray: ", len(compressed[0]))
print("Complete preprocess")

768 768
0  is done
1  is done
2  is done
33619968
11206656
589824
(768, 768)
(768, 768)
(768, 768)
data shape ting (3, 768, 768)
[[43. 46. 50. ... 44. 46. 47.]
 [44. 42. 48. ... 29. 50. 46.]
 [33. 46. 41. ... 44. 45. 46.]
 ...
 [36. 31. 41. ... 48. 57. 46.]
 [37. 30. 33. ... 49. 36. 30.]
 [26. 40. 36. ... 40. 41. 33.]]
Size of spotFinds:  3
spotarray:  768
Complete preprocess


### Image Modification is complete -> data set can be found in spotFinds

In [7]:
compressed = cv2.convertScaleAbs(compressed)#, alpha=(255.0/65535.0))
spotLoc = findSpots(compressed, time_frames, 50) # value indicates background noise average -> tune for spot identification
data = cv2.convertScaleAbs(data)
spotLocLarge = findSpotsPerLayer(data, time_frames, z_stack, 50)
spotLocLarge = np.array(spotLocLarge)
spotLoc = np.array(spotLoc)
#print(spotLoc)
#print(spotLocLarge)
with open('trash.csv', 'w') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(compressed)
writeFile.close()
print("Encontré todos los puntos vivos")
class Spot:
    counts = 0
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z
    def stack_appearance():
        counts += 1
    def appearances(self, counts):
        return self.counts
    
list = []
offset_x = 10 #represents the square around spot container
offset_y = 10
offset_z = 2 # represents distances apart in z stack(zstack is approximately 5 times farther apart than pixels in xy plane)
print(spotLocLarge.shape) # -> 19 time intervals
print(spotLocLarge[0].shape) # -> (number of spots, time intervals prolly) (layers prolly)
print(spotLocLarge[0][0].shape) # -> (spots per layer lel)
for time in range(len(spotLocLarge)):
    for stack in range(len(spotLocLarge[time])):
        for elements in range(len(spotLocLarge[time][stack])): 
            x = spotLocLarge[time][stack][elements][0]
            y = spotLocLarge[time][stack][elements][1]
            list.append(Spot(x,y, stack))
            print(Spot(x, y, stack).x, Spot(x, y, stack).y, Spot(x, y, stack).z)

#print(list)


#returns an object with x any coordinates

(768, 768)
Encontré todos los puntos vivos
(3, 19)
(19,)
(54, 2)
126.0 755.5 0
79.33333587646484 736.6666870117188 0
756.3333129882812 713.6666870117188 0
26.33333396911621 704.3333129882812 0
274.3333435058594 703.6666870117188 0
20.33333396911621 696.3333129882812 0
195.6666717529297 692.6666870117188 0
302.6666564941406 689.6666870117188 0
188.3333282470703 684.3333129882812 0
571.3333129882812 676.3333129882812 0
353.3333435058594 665.6666870117188 0
363.6666564941406 656.6666870117188 0
301.3333435058594 651.3333129882812 0
694.5 634.0 0
245.3333282470703 623.3333129882812 0
602.6666870117188 622.3333129882812 0
761.3333129882812 613.6666870117188 0
334.3333435058594 588.3333129882812 0
295.6666564941406 571.3333129882812 0
754.3333129882812 567.6666870117188 0
211.6666717529297 489.6666564941406 0
385.3333435058594 461.3333435058594 0
404.3333435058594 455.3333435058594 0
20.33333396911621 453.3333435058594 0
230.6666717529297 449.3333435058594 0
296.3333435058594 419.33334350585

349.6666564941406 534.6666870117188 8
275.6666564941406 533.6666870117188 8
295.1111145019531 519.3333129882812 8
519.3333129882812 504.3333435058594 8
243.6666717529297 504.6666564941406 8
690.0 493.5 8
402.5 492.3333435058594 8
279.6666564941406 479.6666564941406 8
732.3333129882812 478.6666564941406 8
446.6666564941406 478.3333435058594 8
430.3333435058594 462.3333435058594 8
232.3333282470703 459.3333435058594 8
474.3333435058594 448.3333435058594 8
722.6666870117188 438.3333435058594 8
641.6666870117188 438.6666564941406 8
189.3333282470703 434.3333435058594 8
643.3333129882812 428.3333435058594 8
249.6666717529297 424.3333435058594 8
369.3333435058594 418.6666564941406 8
125.33333587646484 414.6666564941406 8
326.3333435058594 412.3333435058594 8
518.3333129882812 399.6666564941406 8
611.0 394.6666564941406 8
23.792316436767578 396.5968933105469 8
509.0 383.0 8
267.6666564941406 382.6666564941406 8
649.6666870117188 376.6666564941406 8
639.6666870117188 372.6666564941406 8
503.33

580.0 434.0 12
208.6666717529297 417.6666564941406 12
315.3333435058594 401.6666564941406 12
24.33333396911621 395.6666564941406 12
659.3333129882812 379.3333435058594 12
318.4444580078125 349.1111145019531 12
345.3333435058594 338.3333435058594 12
712.6666870117188 332.6666564941406 12
355.0 330.6666564941406 12
514.111083984375 316.3333435058594 12
716.3333129882812 312.3333435058594 12
449.3333435058594 306.3333435058594 12
699.3333129882812 304.6666564941406 12
451.3333435058594 290.3333435058594 12
530.3333129882812 288.6666564941406 12
703.6666870117188 287.3333435058594 12
240.6666717529297 289.3333435058594 12
254.6666717529297 280.3333435058594 12
444.8888854980469 277.22222900390625 12
437.6666564941406 272.3333435058594 12
282.6666564941406 271.3333435058594 12
601.3333129882812 268.6666564941406 12
37.33333206176758 256.3333435058594 12
605.6666870117188 254.3333282470703 12
153.6666717529297 251.3333282470703 12
598.3333129882812 233.3333282470703 12
625.3333129882812 225.

382.6666564941406 497.3333435058594 15
30.66666603088379 497.6666564941406 15
368.3333435058594 494.3333435058594 15
186.3333282470703 494.6666564941406 15
39.0 491.6666564941406 15
735.3333129882812 490.3333435058594 15
647.6666870117188 488.6666564941406 15
766.6666870117188 486.3333435058594 15
161.6666717529297 485.3333435058594 15
685.6666870117188 482.3333435058594 15
59.66666793823242 481.3333435058594 15
151.3333282470703 479.3333435058594 15
415.8888854980469 472.5555419921875 15
603.5 474.0 15
196.3333282470703 473.6666564941406 15
97.66666412353516 469.6666564941406 15
382.3333435058594 466.3333435058594 15
589.3333129882812 465.3333435058594 15
310.3333435058594 465.3333435058594 15
77.66666412353516 463.6666564941406 15
198.3333282470703 450.6666564941406 15
744.6666870117188 447.3333435058594 15
614.6666870117188 440.3333435058594 15
649.6666870117188 436.3333435058594 15
513.6666870117188 435.3333435058594 15
146.3333282470703 432.0 15
714.6666870117188 430.6666564941406

607.3333129882812 638.6666870117188 17
155.6666717529297 638.3333129882812 17
136.3333282470703 633.6666870117188 17
349.4791564941406 629.5833129882812 17
214.6666717529297 631.3333129882812 17
493.0 626.3333129882812 17
295.3333435058594 625.3333129882812 17
181.6666717529297 624.888916015625 17
414.6666564941406 624.6666870117188 17
175.8333282470703 622.8333129882812 17
62.66666793823242 622.0 17
641.629638671875 615.9629516601562 17
608.3333129882812 613.6666870117188 17
350.8611145019531 607.5555419921875 17
64.33333587646484 609.3333129882812 17
206.6666717529297 607.6666870117188 17
76.66666412353516 603.6666870117188 17
330.3333435058594 600.6666870117188 17
178.0 601.0 17
27.55555534362793 596.111083984375 17
136.6666717529297 591.3333129882812 17
416.6666564941406 589.6666870117188 17
126.66666412353516 590.5555419921875 17
185.6666717529297 588.3333129882812 17
590.6666870117188 581.5555419921875 17
203.0 583.6666870117188 17
454.3333435058594 578.3333129882812 17
264.66665

...

0.3333333432674408 171.3333282470703 16
345.3333435058594 148.6666717529297 16
35.33333206176758 126.33333587646484 16
762.3333129882812 125.33333587646484 16
563.6666870117188 112.66666412353516 16
44.33333206176758 94.66666412353516 16
119.33333587646484 62.33333206176758 16
551.3333129882812 58.66666793823242 16
165.3333282470703 10.666666984558105 16
764.3333129882812 5.333333492279053 16
300.6666564941406 4.666666507720947 16
712.6666870117188 2.6666667461395264 16
108.33333587646484 765.3333129882812 17
27.33333396911621 743.6666870117188 17
145.6666717529297 694.3333129882812 17
403.3333435058594 671.3333129882812 17
188.6666717529297 661.6666870117188 17
277.3333435058594 642.3333129882812 17
623.6666870117188 635.6666870117188 17
558.3333129882812 625.3333129882812 17
263.3333435058594 601.6666870117188 17
321.6666564941406 591.3333129882812 17
0.6666666865348816 579.6666870117188 17
193.3333282470703 541.3333129882812 17
558.3333129882812 527.6666870117188 17
545.333312988281

531.6666870117188 762.6666870117188 4
591.3333129882812 751.3333129882812 4
45.66666793823242 751.3333129882812 4
574.3333129882812 749.3333129882812 4
220.3333282470703 738.6666870117188 4
755.3333129882812 733.6666870117188 4
369.6666564941406 733.3333129882812 4
741.6666870117188 731.3333129882812 4
695.3333129882812 726.3333129882812 4
359.1111145019531 724.5555419921875 4
228.0 713.3333129882812 4
20.0 708.5 4
642.3333129882812 705.6666870117188 4
260.3333435058594 706.6666870117188 4
728.3333129882812 698.6666870117188 4
398.6666564941406 684.3333129882812 4
287.6666564941406 684.3333129882812 4
666.3333129882812 683.3333129882812 4
43.66666793823242 672.3333129882812 4
431.3333435058594 665.6666870117188 4
708.3333129882812 664.3333129882812 4
576.6666870117188 660.3333129882812 4
223.6666717529297 658.3333129882812 4
528.6666870117188 657.3333129882812 4
378.6666564941406 647.6666870117188 4
498.3333435058594 643.3333129882812 4
137.3333282470703 643.6666870117188 4
698.6666870

399.3333435058594 482.6666564941406 7
529.7777709960938 468.3333435058594 7
395.3333435058594 469.3333435058594 7
517.6666870117188 468.6666564941406 7
391.6666564941406 457.6666564941406 7
581.6666870117188 450.6666564941406 7
361.6666564941406 452.0 7
93.33333587646484 450.6666564941406 7
349.6666564941406 443.6666564941406 7
730.3333129882812 438.6666564941406 7
312.3333435058594 437.3333435058594 7
537.6666870117188 435.6666564941406 7
467.3333435058594 433.6666564941406 7
223.6666717529297 434.0 7
206.3333282470703 429.6666564941406 7
165.6666717529297 429.6666564941406 7
355.6666564941406 427.6666564941406 7
591.6666870117188 422.6666564941406 7
484.6666564941406 422.3333435058594 7
197.3333282470703 418.3333435058594 7
694.2222290039062 416.3333435058594 7
583.3333129882812 408.3333435058594 7
169.6666717529297 408.6666564941406 7
7.0 403.5 7
516.625 399.3333435058594 7
141.0 402.3333435058594 7
175.3333282470703 394.0 7
567.6666870117188 387.6666564941406 7
615.0 383.6666564941

290.3333435058594 626.6666870117188 10
500.0 624.3333129882812 10
706.3333129882812 622.6666870117188 10
316.6666564941406 621.3333129882812 10
599.0 617.3333129882812 10
34.11111068725586 616.6666870117188 10
652.3333129882812 616.3333129882812 10
493.6666564941406 615.3333129882812 10
51.33333206176758 607.3333129882812 10
420.6666564941406 604.6666870117188 10
309.3333435058594 602.6666870117188 10
204.6666717529297 603.6666870117188 10
431.1666564941406 597.6666870117188 10
612.6666870117188 596.3333129882812 10
645.6666870117188 594.6666870117188 10
274.0 591.5 10
489.5 591.0 10
124.33333587646484 588.3333129882812 10
563.6666870117188 583.3333129882812 10
547.3333129882812 578.6666870117188 10
758.3333129882812 574.3333129882812 10
606.6666870117188 573.6666870117188 10
576.6666870117188 570.6666870117188 10
175.6666717529297 568.6666870117188 10
265.6666564941406 567.3333129882812 10
550.3333129882812 566.3333129882812 10
339.4444580078125 557.2222290039062 10
241.6666717529297 

589.6666870117188 132.3333282470703 12
679.6666870117188 125.66666412353516 12
649.6666870117188 125.66666412353516 12
376.3333435058594 125.33333587646484 12
665.3333129882812 123.66666412353516 12
413.40740966796875 121.29629516601562 12
618.6666870117188 114.66666412353516 12
673.0 113.33333587646484 12
4.333333492279053 112.33333587646484 12
400.3333435058594 110.66666412353516 12
415.3333435058594 106.66666412353516 12
185.6666717529297 104.66666412353516 12
543.5555419921875 101.44444274902344 12
621.6666870117188 99.66666412353516 12
561.3333129882812 94.33333587646484 12
160.6666717529297 92.66666412353516 12
204.3333282470703 91.66666412353516 12
396.6666564941406 91.33333587646484 12
669.6666870117188 89.33333587646484 12
538.3333129882812 89.66666412353516 12
273.6666564941406 83.66666412353516 12
17.66666603088379 81.33333587646484 12
524.25927734375 69.70370483398438 12
650.3333129882812 62.66666793823242 12
366.6666564941406 65.33333587646484 12
108.33333587646484 59.3333

In [8]:
#identify spots based on occurrence in a region
def checkExisting(Spot1, Spot2):
    #print(Spot1.x, Spot2.x)
    if(np.abs(Spot1.x - Spot2.x) < offset_x and 
       np.abs(Spot1.y - Spot2.y) < offset_y and 
       np.abs(Spot1.z - Spot2.z) < offset_z):
        Spot1.stack_appearance
spotlist = list.copy()
for element in spotlist:
    for element1 in spotlist:
        if(element == element1):
            continue
        checkExisting(element, element1)

print("Completed")

Completed


In [72]:
#appearances = spotlist.sort(key = lambda x: x.appearances, reverse = True)
#print(spotlist[:10].appearances)

#list(map(x, spotlist[:10]))
attributes = [obj.appearances for obj  in spotlist[:10]]
print(attributes)


[<bound method Spot.appearances of <__main__.Spot object at 0x1a282f6cf8>>, <bound method Spot.appearances of <__main__.Spot object at 0x1a2a7f5e48>>, <bound method Spot.appearances of <__main__.Spot object at 0x1a2a7f5128>>, <bound method Spot.appearances of <__main__.Spot object at 0x1a2a7f5b00>>, <bound method Spot.appearances of <__main__.Spot object at 0x1a2911c780>>, <bound method Spot.appearances of <__main__.Spot object at 0x1a2911ca90>>, <bound method Spot.appearances of <__main__.Spot object at 0x1a2911c0b8>>, <bound method Spot.appearances of <__main__.Spot object at 0x1a2911cdd8>>, <bound method Spot.appearances of <__main__.Spot object at 0x1a2911cba8>>, <bound method Spot.appearances of <__main__.Spot object at 0x1a290bb9e8>>]


In [ ]:

print(data[0,0].shape)
print(type(data[0,0]))
arr = data[0,0]
arr = (arr).reshape(768, 768)

In [ ]:
n = 20
for i in range(n):
    print(int(int(n/2) -np.absolute(np.round(n/2)-i-1)))

In [1]:
who

NamespaceMagics	 get_ipython	 getsizeof	 json	 np	 var_dic_list	 
